In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.experimental import enable_iterative_imputer as poop
import pandas as pd
import numpy as np
import random


In [4]:
table = pd.read_csv("train.csv")
table2 = pd.read_csv("test.csv")

In [6]:
df = table.drop(columns=["PassengerId", "Name", "Cabin"])
df2 = table2.drop(columns=["PassengerId", "Name", "Cabin"])
df.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [13]:
#age imputation using iterativeImpute
age = df.copy()
model = poop(random_state=0)
model.transform(age["Age"])
print(age["Age"])


TypeError: 'module' object is not callable

In [4]:
#Converting single column of bool values to column of ints 1 or 0
#Transported 0 = False, 1 = True
int_df = df.copy()
int_df["Transported"] = int_df["Transported"].astype(int)

int_df.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,0
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,1
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,0
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,0
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,1


In [14]:
#converting NaN values in RoomService, FoodCourt, ShoppingMall, Spa, VRDeck to avg
services_df = df.copy()
services_df = services_df.drop(columns=["HomePlanet", "CryoSleep", "Destination", "Age", "VIP", "Transported"])
print(services_df["RoomService"].mean())
for col in services_df.columns[services_df.isnull().any(axis=0)]:
    services_df[col].fillna(services_df[col].mean(), inplace=True)
services_df.round(1)
print(services_df)

224.687617481203
      RoomService  FoodCourt  ShoppingMall     Spa  VRDeck
0             0.0        0.0           0.0     0.0     0.0
1           109.0        9.0          25.0   549.0    44.0
2            43.0     3576.0           0.0  6715.0    49.0
3             0.0     1283.0         371.0  3329.0   193.0
4           303.0       70.0         151.0   565.0     2.0
...           ...        ...           ...     ...     ...
8688          0.0     6819.0           0.0  1643.0    74.0
8689          0.0        0.0           0.0     0.0     0.0
8690          0.0        0.0        1872.0     1.0     0.0
8691          0.0     1049.0           0.0   353.0  3235.0
8692        126.0     4688.0           0.0     0.0    12.0

[8693 rows x 5 columns]


In [6]:
#OneHotEncoding the dataframe
tempUnique = []
for value in df["HomePlanet"]:
    if value not in tempUnique:
        tempUnique.append(value)

print(tempUnique)


['Europa', 'Earth', 'Mars', nan]


In [8]:
#Total 8693 Rows (Excluding top Row)
#IN "HomePlanet"
#Earth  4602 // 52.95% 
#Europa 2131 // 24.52%
#Mars   1759 // 20.24%
#Nan    200  //  2.3%

#-----200(.5295)+200(.2452)+200(.2024) = 195.42 ~ 195

#Nan Distribution
#-------------------
#Earth get 107 additional
#Europa get 51 additional 
#Mars get 42 additional

remove_nans_df = df.copy()

#Dictionary to distribute values
home_planet_dict = {"Earth": 107, "Mars":43, "Europa":51}
cyrosleep_dict = {True: 109, False: 108}
vip_dict = {True: 101, False: 102}

#Function to get random balue from dictionary and update it
def get_random_dict_value_and_update(dict):
    if not dict:
        return np.nan
    key = random.choice(list(dict.keys()))
    dict[key] -= 1
    if dict[key] == 0:
        del dict[key]
    return key

#Replace each NaN in 'HomePlanet' with a random value from home planet dict
remove_nans_df['HomePlanet'] = remove_nans_df["HomePlanet"].apply(lambda x: get_random_dict_value_and_update(home_planet_dict) if pd.isnull(x) else x)
remove_nans_df['CryoSleep'] = remove_nans_df["CryoSleep"].apply(lambda x: get_random_dict_value_and_update(cyrosleep_dict) if pd.isnull(x) else x)
remove_nans_df['VIP'] = remove_nans_df["VIP"].apply(lambda x: get_random_dict_value_and_update(vip_dict) if pd.isnull(x) else x)

print(remove_nans_df["HomePlanet"].value_counts())
print(remove_nans_df["CryoSleep"].value_counts())
print(remove_nans_df["VIP"].value_counts())
#T


HomePlanet
Earth     4709
Europa    2182
Mars      1802
Name: count, dtype: int64
CryoSleep
False    5547
True     3146
Name: count, dtype: int64
VIP
False    8393
True      300
Name: count, dtype: int64


In [31]:
#logistic regression on table. 1by1 or table...still debating
df2_services = services_df
clf = LogisticRegression(random_state=0)
y = df["Transported"]
fitted_model = clf.fit(df2_services, y)
x_test = df2.drop(columns=["HomePlanet", "CryoSleep", "Destination", "Age", "VIP"])
#print(x_test)
predictions = fitted_model.predict(x_test)
print(classification_report(x_test, predictions))

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values